In [12]:
import pandas as pd 
import numpy as np

In [17]:
raw_business_data = pd.read_csv('Legally_Operating_Businesses_10272017.csv',index_col = 'LICENSE_NBR')
raw_nyc_data = pd.read_csv('new_york.csv')

In [18]:
df_b = raw_business_data[raw_business_data['STATE'] == 'NY']

In [22]:
df_b.columns
raw_nyc_data

,Zip Code,City,State,Population,Population Estimate,Business Annual Payroll,Business First Quarter Payroll,Median Age,Male Population,Female Population,...,Black or African American Population,American Indian or Alaskan Native Population,Native Hawaiian or other Pacific Islander Population,White Population,Other Population,Growing Counties A,Growing Counties B,Growth Increase Number,Growth Increase Percentage,Growth Rank
0,12007,ALCOVE,NY,60,157,851,152,47.7,35,25,...,1,1,0,58,0,0,0,0,0,0
1,12009,ALTAMONT,NY,7164,6920,76219,16009,44.5,3570,3594,...,123,31,1,6900,70,0,0,0,0,0
2,12023,BERNE,NY,2196,2115,1472,262,44.6,1119,1077,...,20,5,0,2173,7,0,0,0,0,0
3,12041,CLARKSVILLE,NY,481,172,2081,416,42.7,256,225,...,4,1,0,475,3,0,0,0,0,0
4,12045,COEYMANS,NY,584,0,3640,893,34.2,288,296,...,30,13,0,563,8,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2148,14527,PENN YAN,NY,12820,13688,105635,24366,42.9,6217,6603,...,163,72,6,12593,88,0,0,0,0,0
2149,14544,RUSHVILLE,NY,2101,2067,6660,1576,41.2,1036,1065,...,15,10,0,2076,9,0,0,0,0,0
2150,14837,DUNDEE,NY,5648,5961,18087,3717,39.5,2802,2846,...,69,37,8,5541,27,0,0,0,0,0
2151,14842,HIMROD,NY,938,978,909,80,35.4,471,467,...,4,0,0,936,0,0,0,0,0,0


In [20]:
df_business = df_b[['LIC_EXPIR_DD','LICENSE_STATUS', 'LICENSE_CREATE_DATE','INDUSTRY','ZIP','B1_X_COORD', 'B1_Y_COORD']]

In [21]:
df_business = df_business.dropna()